# Plasmo.jl for DCOPF

* Centralized way

* Load data

In [1]:
using LightGraphs, Random, LinearAlgebra, Printf, DelimitedFiles, SparseArrays, PowerModels

In [2]:
PowerModels.silence()

[info | PowerModels]: Suppressing information and warning messages for the rest of this session.  Use the Memento package for more fine-grained control of logging.


In [3]:
function get_data(path)
    data = PowerModels.parse_file(path)
    A = PowerModels.calc_susceptance_matrix(data)

    args = Dict()
    args[:N] = size(A.matrix, 1) # number of buses
    args[:y] = A.matrix
    args[:del] = spzeros(args[:N],args[:N]) # 
    args[:g] = LightGraphs.Graph(args[:N]) # the lightgraph; assign the nodes
    for e in values(data["branch"])
        i = A.bus_to_idx[e["f_bus"]]
        j = A.bus_to_idx[e["t_bus"]]
        LightGraphs.add_edge!(args[:g],i,j) # assign the edges.
        args[:del][i,j] = e["angmin"]
        args[:del][j,i] = e["angmin"]
    end

    # voltage angle limits
    args[:val] = -ones(args[:N])*Inf # lower bound
    args[:vau] = ones(args[:N])*Inf # upper bound
    args[:ref] = A.bus_to_idx[PowerModels.reference_bus(data)["index"]] # reference buses
    args[:val][args[:ref]] = 0; args[:vau][args[:ref]] = 0 # the voltage angle of reference bus is 0.


    gens = PowerModels.bus_gen_lookup(data["gen"], data["bus"]) # bus index 중 generator bus index에 대해서만 관련 PV bus 정보 제공하는 함수. (vg, mbase, source_id, pg, model, shutdown, startup, index, cost, qg)
    # generator cost coefficients
    args[:c1] = [[1e6, -1e6] for i=1:args[:N]]
    args[:c2] = [[.0, .0] for i=1:args[:N]]
    args[:sl] = [[.0,-1e2] for i=1:args[:N]] # generator power min?
    args[:su] = [[1e2,.0] for i=1:args[:N]] # generator power max?
    args[:ng] = 2*ones(Int64,args[:N]) # number of generators? <= TODO: Need to check this.

    for i=1:args[:N] # for each bus
        bus = A.idx_to_bus[i] # return to the bus index from the index of matrix.
        for j=1:length(gens[bus]) # 해당 PV bus에 있는 발전기 개수 별 loop?
            if length(gens[bus][j]["cost"]) != 0
                # push! <= list append와 같은 개념
                push!(args[:c1][i], gens[bus][j]["cost"][1])
                push!(args[:c2][i], gens[bus][j]["cost"][2])
                push!(args[:sl][i], gens[bus][j]["pmin"])
                push!(args[:su][i], gens[bus][j]["pmax"])
                args[:ng][i] += 1
            end
        end
    end

    args[:Ng] = sum(args[:ng]) # <= TODO: Need to check this.

    # PQ bus의 Pd 저장?
    args[:sd] = zeros(args[:N])
    for v in values(data["load"])
        args[:sd][A.bus_to_idx[v["load_bus"]]] = v["pd"]
    end

    return args
end

get_data (generic function with 1 method)

In [5]:
path = "/home/kjsong/workspace/Plasmo_tutorial/data/pglib_opf_case9241_pegase.m"
# "/home/kjsong/workspace_skj/Plasmo_tutorial/data/pglib_opf_case9241_pegase.m"
# /home/kjsong/workspace/Plasmo_tutorial/data/pglib_opf_case9241_pegase.m
args = get_data(path)

Dict{Any, Any} with 14 entries:
  :su  => [[100.0, 0.0], [100.0, 0.0, 1.0], [100.0, 0.0], [100.0, 0.0], [100.0,…
  :vau => [Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf  …  Inf, Inf, Inf, …
  :c1  => [[1.0e6, -1.0e6], [1.0e6, -1.0e6, 4814.49], [1.0e6, -1.0e6], [1.0e6, …
  :c2  => [[0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.…
  :ng  => [2, 3, 2, 2, 2, 3, 2, 3, 2, 2  …  2, 2, 2, 3, 2, 2, 3, 3, 2, 2]
  :N   => 9241
  :val => [-Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf  …  -Inf,…
  :ref => 4231
  :g   => {9241, 14207} undirected simple Int64 graph
  :sd  => [3.784, 0.0, 1.51, 1.7141, 0.4595, 0.0, 0.3103, 0.0, 0.143, 1.34  …  …
  :y   => sparse([1, 2316, 7639, 7835, 2, 2447, 4898, 3, 8189, 8676  …  9237, 1…
  :del => sparse([2316, 7639, 7835, 2447, 4898, 8189, 8676, 1813, 5241, 9018  ……
  :Ng  => 19927
  :sl  => [[0.0, -100.0], [0.0, -100.0, -1.0853], [0.0, -100.0], [0.0, -100.0],…

* Powergrid
    * How to model the DCOPF in terms of graph structure using Plasmo.jl (**Most important part**)
    * TODO: ACOPF로 문제 변경 시 달라지는 점이 무엇인지 파악

In [6]:
using Plasmo

In [7]:
# 뭐에 대한 parameter?
gamma = 1e5 # the regularization parameter of voltage angle difference. 
oms = [1, 4, 7]
max_iter = 1000
scl = 1

g = args[:g] # LightGraphs type
N_buses = LightGraphs.nv(g) # get the number of nodes in graph.
N_lines = LightGraphs.ne(g) # get the number of edges in graph.
y = args[:y] # get the Y bus matrix 
del = args[:del] # get the voltage angle difference lower bound matrix

powergrid = Plasmo.OptiGraph()
# 여기가 MAIN PART. i.e., power network 최적화 문제를 graph로 어떻게 모델링 하는지.
Plasmo.@optinode(powergrid, buses[1:N_buses]) # create node buses
Plasmo.@optinode(powergrid, lines[1:N_lines]) # create transmission lines <= 일반적으로 power network의 line은 edge로 생각하겠지만, 여기서는 최적화 문제를 graph로 표현할때 line에 대한 variable (branch power flow)이 있기 때문에 node로 모델링.

# 각 bus별로 from bus인지 to bus인지 각 line에 따라 정해져 있는데, 이를 bus 관점에서 line을 바라보는, 즉 어떤 line이 bus에 대해서 들어오는 line인지 아님 나가는 line인지를 볼 수 있는 dict.
node_map_in = Dict((bus, Plasmo.OptiNode[]) for bus in buses) 
node_map_out = Dict((bus, Plasmo.OptiNode[]) for bus in buses)

line_map = Dict() # 각 transmission line에 대해 연결된 bus pair; element type이 graph의 node로 모델링된 최적화 변수들임.
edge_map = Dict() # 각 transmission line이 어떤 bus pair에 해당되어 있는지에 대한 dict.
B = Dict() # 각 transmission line의 susceptance
angle_rate = Dict() # 각 transmission line의 bus pair에 대한 voltage angle difference limit
ngens = Dict()
load_map = Dict()

for (i,edge) in enumerate(LightGraphs.edges(g))
    line = lines[i]
    v_from = edge.src # <= LightGraphs package의 method: edge에 해당하는 node들 중 source node 불러오기
    v_to = edge.dst # <= LightGraphs package의 method: edge에 해당하는 node들 중 destination node 불러오기

    edge_map[(v_from, v_to)] = line

    B[line] = y[v_from,v_to]
    angle_rate[line] = del[v_from,v_to]

    bus_from = buses[v_from]
    bus_to = buses[v_to]
    bus_vec = [bus_from, bus_to] # vector form
    line_map[line] = bus_vec 

    push!(node_map_in[bus_to], line)
    push!(node_map_out[bus_from], line)
end

for i = 1:N_buses
    neighs = LightGraphs.neighbors(g,i)
    bus = buses[i]

    ngens[bus] = args[:ng][i] # 이건 뭘 의미하는건지...
    load_map[bus] = args[:sd][i] # Bus 별 Pd 값 저장

    # Plasmo OptiNode(i.e., power network의 bus 및 line)에 변수 및 제약조건 설정에 필요한 값 할당.
    bus.ext[:c1] = args[:c1][i] 
    bus.ext[:c2] = args[:c2][i]
    bus.ext[:va_lower] = args[:val][i]
    bus.ext[:va_upper] = args[:vau][i]
    bus.ext[:gen_lower] = args[:sl][i]
    bus.ext[:gen_upper] = args[:su][i]
end

for line in lines
    bus_from = line_map[line][1]
    bus_to = line_map[line][2]
    Plasmo.@variable(line, va_i, start=0)
    Plasmo.@variable(line, va_j, start=0)
    Plasmo.@variable(line, flow, start=0) # i.e., branch power flow
    Plasmo.@constraint(line, flow==B[line]*(va_i - va_j)) # power flow equation constraint.
    delta = angle_rate[line]
    Plasmo.@constraint(line, delta <= (va_i - va_j) <= -delta) # voltage angle difference constraint.
    Plasmo.@objective(line, Min, 1/4*gamma*(va_i - va_j)^2) # penalty term of objective function (18a).
end

dual_links = Plasmo.LinkConstraintRef[] # link constraints들을 저장하고 관리할 수 있는 storage 개념의 리스트.
# primal_links = Plasmo.LinkConstraintRef[]
for (i,bus) in enumerate(buses)
    va_lower = bus.ext[:va_lower]
    va_upper = bus.ext[:va_upper]

    gen_lower = bus.ext[:gen_lower]
    gen_upper = bus.ext[:gen_upper]

    Plasmo.@variable(bus, va_lower <= va <= va_upper, start=0) # voltage angle variable.
    # TODO: 아래 변수가 의미하는 것이 뭔지 이해 필요!
    # 일단 active power인 건 알겠음.. => Gen이 있으면 3종류의 active power, 없으면 2종류의 active power?
    Plasmo.@variable(bus, P[j=1:ngens[bus]], start=0)
    for j=1:length(P)
        Plasmo.set_lower_bound(P[j],gen_lower[j])
        Plasmo.set_upper_bound(P[j],gen_upper[j])
    end
    # NOTE: 위에 loop에서 transmission line node에 대한 변수 및 제약 조건을 설정해줬기 때문에, 자동으로 node_map_in에 있는 value 값들도 동기화가 됨!
    lines_in = node_map_in[bus]
    lines_out = node_map_out[bus]

    # NOTE: DC/AC OPF에서의 link constraints 선언 방식!! <= 다른 방식은 없는지?
    Plasmo.@variable(bus, power_in[1:length(lines_in)])
    Plasmo.@variable(bus, power_out[1:length(lines_out)])

    for (j,line) in enumerate(lines_in)
        link = Plasmo.@linkconstraint(powergrid, bus[:power_in][j] == line[:flow])
        push!(dual_links,link)
    end
    for (j,line) in enumerate(lines_out)
        link = Plasmo.@linkconstraint(powergrid, bus[:power_out][j] == line[:flow])
        push!(dual_links,link)
    end


    ## Julia error: sum of the empty vector is not zero...
    if typeof(power_in) == Vector{Any}
        # bus[:power_in] = 0
        power_in = 0
    end
    if typeof(power_out) == Vector{Any}
        # bus[:power_out] = 0
        power_out = 0
    end
    Plasmo.@constraint(bus, power_balance, sum(bus[:P][j] for j=1:ngens[bus]) + sum(power_in) - sum(power_out) - load_map[bus] == 0) # power balance equation constraint.
    Plasmo.@objective(bus, Min, sum(bus.ext[:c1][j]*bus[:P][j] + bus.ext[:c2][j]*bus[:P][j]^2 for j=1:ngens[bus]))
end

Plasmo.@linkconstraint(powergrid, line_coupling_i[line = lines], line[:va_i] == line_map[line][1][:va])
Plasmo.@linkconstraint(powergrid, line_coupling_j[line = lines], line[:va_j] == line_map[line][2][:va])
primal_links = [line_coupling_i.data ; line_coupling_j.data]

28414-element Vector{LinkConstraintRef}:
 line_coupling_i[OptiNode w/ 3 Variable(s) and 2 Constraint(s)]: lines[1][:va_i] - buses[1][:va] = 0
 line_coupling_i[OptiNode w/ 3 Variable(s) and 2 Constraint(s)]: lines[2][:va_i] - buses[1][:va] = 0
 line_coupling_i[OptiNode w/ 3 Variable(s) and 2 Constraint(s)]: lines[3][:va_i] - buses[1][:va] = 0
 line_coupling_i[OptiNode w/ 3 Variable(s) and 2 Constraint(s)]: lines[4][:va_i] - buses[2][:va] = 0
 line_coupling_i[OptiNode w/ 3 Variable(s) and 2 Constraint(s)]: lines[5][:va_i] - buses[2][:va] = 0
 line_coupling_i[OptiNode w/ 3 Variable(s) and 2 Constraint(s)]: lines[6][:va_i] - buses[3][:va] = 0
 line_coupling_i[OptiNode w/ 3 Variable(s) and 2 Constraint(s)]: lines[7][:va_i] - buses[3][:va] = 0
 line_coupling_i[OptiNode w/ 3 Variable(s) and 2 Constraint(s)]: lines[8][:va_i] - buses[4][:va] = 0
 line_coupling_i[OptiNode w/ 3 Variable(s) and 2 Constraint(s)]: lines[9][:va_i] - buses[4][:va] = 0
 line_coupling_i[OptiNode w/ 3 Variable(s) and 2 C

* Solve in a Centralized way

In [30]:
using Ipopt, Gurobi, JuMP
using MadNLP, MadNLPHSL

import HSL_jll

In [31]:
# JuMP.set_optimizer(powergrid, Ipopt.Optimizer)
# JuMP.set_optimizer(powergrid, ()->MadNLP.Optimizer(linear_solver=Ma57Solver))
JuMP.set_optimizer(powergrid, Gurobi.Optimizer)

JuMP.optimize!(powergrid)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2439995
Academic license 2439995 - for non-commercial use only - registered to kj___@sogang.ac.kr
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2439995 - for non-commercial use only - registered to kj___@sogang.ac.kr
Optimize a model with 94483 rows, 114410 columns and 247239 nonzeros
Model fingerprint: 0x3231521f
Model has 42621 quadratic objective terms
Coefficient statistics:
  Matrix range     [8e-03, 2e+04]
  Objective range  [1e+02, 1e+06]
  QObjective range [5e+04, 1e+05]
  Bounds range     [8e-03, 1e+02]
  RHS range        [1e-04, 3e+01]
Presolve removed 51871 rows and 51872 columns
Presolve time: 0.21s
Presolved: 42612 rows, 62538 columns, 157689 nonzeros
Presolved model has 

In [32]:
cost = JuMP.objective_value(powergrid)

6.8011041591796875e6

In [33]:
solution = Dict(
    JuMP.name(v) => JuMP.value(v) for v in JuMP.all_variables(powergrid)
    )

Dict{String, Float64} with 100203 entries:
  "buses[8708][:P[2]]"         => -7.10543e-14
  "lines[2645][:va_j]"         => -0.639085
  "lines[1026][:flow]"         => -0.0329363
  "buses[5112][:va]"           => -0.556671
  "buses[8626][:P[2]]"         => -7.10543e-14
  "buses[4056][:power_out[1]]" => -1.0253
  "lines[3636][:va_j]"         => -0.55959
  "lines[12512][:va_j]"        => -0.830198
  "buses[7387][:P[2]]"         => -7.10543e-14
  "lines[8070][:flow]"         => 0.575689
  "buses[6657][:power_out[2]]" => -5.26938
  "lines[2573][:va_j]"         => -0.461642
  "buses[8301][:P[1]]"         => 6.59087e-14
  "lines[6802][:va_i]"         => -0.941918
  "buses[7230][:power_in[1]]"  => -0.0018
  "lines[9261][:va_j]"         => -0.817538
  "buses[1941][:power_out[3]]" => 2.95853
  "buses[7904][:power_in[1]]"  => 1.72608
  "buses[5600][:power_in[1]]"  => 0.0944315
  ⋮                            => ⋮